In [1]:
from prefect import flow,task
from pathlib import Path
import pandas as pd
from prefect_gcp.cloud_storage import GcsBucket
import requests
from datetime import datetime,timedelta

In [2]:
fromdate=(datetime.now()-timedelta(days=1)).date()
baseurl=f"https://api.openaq.org/v2/measurements?date_from={fromdate}T00%3A00%3A00&date_to={fromdate}T23%3A59%3A59&limit=10000&sort=desc&country=IN&radius=25000&order_by=datetime"
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}
response=requests.get(baseurl,headers=headers)
print(response.status_code)
if response.status_code==200:
    data=response.json()
    # print(data['data'])
else:
    print("fail")

200


In [3]:
df=pd.json_normalize(data['results'])
print(df.dtypes)
# df = df.explode('measurements', ignore_index=True)
print(df.head(-5))
print(df.size)
# Normalize the exploded 'measurements' column to create separate columns for each measurement attribute


locationId                 int64
location                  object
parameter                 object
value                    float64
unit                      object
country                   object
city                      object
isMobile                    bool
isAnalysis                object
entity                    object
sensorType                object
date.utc                  object
date.local                object
coordinates.latitude     float64
coordinates.longitude    float64
dtype: object
      locationId                             location parameter       value  \
0         273205                                UT133     um003    0.000000   
1          73924                         US Embassy B     um003   37.966364   
2          64934                        NASA_AQCS_123  humidity   53.000000   
3         354180                           HaryanaAQI      pm25   15.592857   
4          64930                        NASA_AQCS_109     um050    0.000000   
...          ... 

In [4]:
changetype={'location':str, 'parameter':str, 'unit':str,
                'country':str, 'city':str, 'entity':str, 
                'sensorType':str
                }
df=df.astype(changetype)
df['date.utc']=pd.to_datetime(df['date.utc'])
df['date.local']=df['date.local'].str[:19]
df['date.local']=pd.to_datetime(df['date.local'])
print(df.dtypes)
print(df.head())

locationId                             int64
location                              object
parameter                             object
value                                float64
unit                                  object
country                               object
city                                  object
isMobile                                bool
isAnalysis                            object
entity                                object
sensorType                            object
date.utc                 datetime64[ns, UTC]
date.local                    datetime64[ns]
coordinates.latitude                 float64
coordinates.longitude                float64
dtype: object
   locationId       location parameter      value           unit country  \
0      273205          UT133     um003   0.000000  particles/cm³      IN   
1       73924   US Embassy B     um003  37.966364  particles/cm³      IN   
2       64934  NASA_AQCS_123  humidity  53.000000              %      IN   
3      

In [6]:
print(df.describe())

         locationId        value  coordinates.latitude  coordinates.longitude
count  4.771000e+03  4771.000000           4771.000000            4771.000000
mean   2.775287e+05   102.616809             23.315422              67.634154
std    3.786690e+05   253.275127              7.910121              47.200466
min    8.039000e+03     0.000000             12.448359            -121.979836
25%    6.667300e+04     0.093667             13.048528              75.722360
50%    2.202660e+05    13.334483             27.310116              77.573395
75%    2.732050e+05    64.934872             28.598204              78.214710
max    1.667903e+06  1016.395517             37.298252              88.596820


In [19]:
df=df.drop(columns=['isAnalysis'])

In [21]:
df.isna().sum().sum()

0